## Notification server

The flow of events that correspond to the ZTP phases can be monitored using NETCONF notifications.
NSO is configured to expose notifications when an action has been run or a state has changed. Using a netconf client (ncclient for python, anc/jnc for java, etc) we can connect to NSO over NETCONF and listen indefinitely to incoming updates for a stream that tracks the configuration retrieval and onboarding steps of ZTP. On reception of a notification, the application makes a REST call that will result in posting the notification in a Webex Teams chat-room on behalf of a chat-bot.

The implementation of this NETCONF/REST application is written in Python v3 with ncclient a NETCONF client library and Requests as an HTTP library. The application runs on the NSO host and it listens to incoming NETCONF notifications. 

### Post notification to Webex teams method

The notifications are posted to the chat-room named ztp-vt, on behalf of a chat-bot named ZTP-VT, using the REST API of Webex. The POST operation requires the following:

•	Notification as payload

•	Token that identifies the chat-bot

•	Id that identifies the chat-room


In [2]:
from ncclient import manager
import requests
import json

def post_notification(notification, bot_token, room_id):
    header = {"Authorization": "Bearer %s" % bot_token,
              "Content-Type": "application/json"}

    payload = {"roomId": room_id,
               "markdown": "```" + notification}

    result = requests.post("https://api.ciscospark.com/v1/messages/",
                           headers=header,
                           data=json.dumps(payload),
                           verify=True)

    if result.status_code == 200:
        print("Successfully posted to Webex Teams room")
    else:
        print("Failed to post to Webex Teams room with status code: %s" % result.status_code)
        if result.status_code == 400:
            print("Check that the room id is correct")
        elif result.status_code == 401:
            print("Check that the bot token is correct")
        elif result.status_code == 404:
            print("Check that the bot is part of the room")


ModuleNotFoundError: No module named 'ncclient'

### Listen for incoming notifications

NSO uses its Northbound to expose a custom stream of NETCONF notifications called ztp. Each time NSO onboards a device, it sends a notification on this stream. The subscriber can then fetch this notification and pass it further to the chat-room. 
We use ncclient to subscribe to the ztp stream, and every time a notification is received, we post it to the Webex teams room

In [1]:
if __name__ == '__main__':

    # Read NSO credentials
    session = {}
    # NSO credentials
    session['nso'] = {}
    session['nso']['host'] = "198.18.134.50"
    session['nso']['port'] = "2022"
    session['nso']['user'] = "admin"
    session['nso']['pass'] = "admin"

    session['stream'] = "ztp"
    session['teams_bot_token'] = "ZjczZjFmYTctNDkwNy00ZjBlLTljZjQtN2QyNTQ2YzJkMzAwNGMwNzM1OTktNTZl_PF84_1eb65fdf-9643-417f-9974-ad72cae0e10f"
    session['teams_room_id'] = "Y2lzY29zcGFyazovL3VzL1JPT00vNzRiZDE2MjAtNDFiMC0xMWVhLTgzZjUtMGZjZWJmNWQ3OTlm"
    

    # Connect to NSO
    m = manager.connect(look_for_keys=False,
                         host=session['nso']['host'],
                         port=session['nso']['port'],
                         username=session['nso']['user'],
                         password=session['nso']['pass'],
                         hostkey_verify=False)

    try:
        # Subscribe to notifications of a stream
        m.create_subscription(stream_name=session['stream'])

    except Exception as e:
        print('Failed to create subscription:', e)
    while True:
        # Wait indefinitely for notifications
        notification = m.take_notification()

        # Print notification
        print(notification.notification_xml)

        # Send notification to the Webex Teams room
        post_notification(notification.notification_xml,
                          session['teams_bot_token'],
                          session['teams_room_id'])
    m.close_session()
    
    

NameError: name 'manager' is not defined